In [1]:
from __future__ import print_function, division
import os, random, time, copy
import numpy as np
import pandas as pd
import os.path as path
import scipy.io as sio
from scipy import misc
from scipy import ndimage, signal
import scipy
import pickle
import sys
import math
import matplotlib.pyplot as plt
from io import BytesIO


import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler 
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import torchvision.utils as vutils


import warnings 
warnings.filterwarnings("ignore")
print(sys.version)
print(torch.__version__)


manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

3.6.13 |Anaconda, Inc.| (default, Feb 23 2021, 21:15:04) 
[GCC 7.3.0]
1.8.0+cu111
Random Seed:  999


In [2]:
torch.manual_seed(0)

device ='cpu'
if torch.cuda.is_available(): 
    device='cuda:0'


batch_size = 16    
bestEpoch = 4  
lr = 0.0001 
num_epochs = 5
torch.cuda.device_count()
torch.cuda.empty_cache()

save_dir="./savedata/RPGAN"

print(save_dir)    
if not os.path.exists(save_dir): os.makedirs(save_dir)

log_filename = os.path.join(save_dir, 'train.log')

./savedata/RPGAN


In [3]:

#Fourier transform

import librosa

def changesy(xtrain0):

    sn=np.array(xtrain0)
    sy=xtrain0.unsqueeze(1)
    X = librosa.stft(sn, n_fft=52, hop_length=64)
    a=np.abs(X)
    b=np.angle(X)
    c=[]
    c.append(a)
    c.append(b)
    c.append(sy)
    c=np.concatenate(c)
    
    return c
    

def changeX(xtrain):
    yt=[]
    countx=len(xtrain)
    print("len:",countx)
    for i in range(countx):
        yt.append(changesy(xtrain[i]))
        qt=np.array(yt)
        qt=torch.tensor(qt)
        qt=torch.squeeze(qt)
    return qt
        

In [4]:

print('==> Preparing data.............................')


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.datasets import MNIST, CIFAR10, CIFAR100, SVHN

import os
import torch
from torch import nn,optim
import torch.nn.functional as F

from torchvision import datasets, transforms

from time import perf_counter

import  numpy as np
import torch.utils.data as Data

from torch.utils.data import Dataset, DataLoader

class Safeman(Dataset):
    
    def __init__(self, data,targets):
        super(Safeman, self).__init__()
        self.data = data
        self.targets = targets
        
     
    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        img, target = self.data[idx], self.targets[idx]
        return img, target

class Safeman_Filter(Safeman):

    def __Filter__(self, known):
        targets = self.targets.data.numpy()
        mask, new_targets = [], []
        for i in range(len(targets)):
            if targets[i] in known:
                mask.append(i)
                new_targets.append(known.index(targets[i]))
        self.targets = np.array(new_targets)
        mask = torch.tensor(mask).long()
        self.data = torch.index_select(self.data, 0, mask)
        
class Safeman_FilterB(Safeman):

    def __Filter__(self, known):
        targets = self.targets.data.numpy()
        new_targets = []
        for i in range(len(targets)):
            if targets[i] in known:
                new_targets.append(0)
            else:
                new_targets.append(1)
        self.targets = np.array(new_targets)
        self.data = self.data

class Safeman_FilterC(Safeman):
    
    def __Filter__(self, trainknown):
        train_class_num=len(trainknown)
        for i in range(0,len(self.targets)) :
            if self.targets[i]>train_class_num:
                self.targets[i] = train_class_num
        self.data = self.data

class Safeman_FilterD(Safeman):

    def __Filter__(self, known):
        targets = self.targets.data.numpy()
        mask, new_targets = [], []
        train_class_num=len(known)
        for i in range(len(targets)):
            if targets[i] not in known:
                mask.append(i)
                new_targets.append(train_class_num)
        self.targets = np.array(new_targets)
        mask = torch.tensor(mask).long()
        self.data = torch.index_select(self.data, 0, mask)




known=[0, 1, 2,3,4]
unknown=[ 5,6,7]

num_class=len(known)



X_train0 = np.load('./dataset/X_train_ukm.npy')
y_train1 = np.load('./dataset/y_train_ukm.npy')
X_final_test0 = np.load('./dataset/X_final_test_ukm.npy' )
y__final_test1 = np.load('./dataset/y__final_test_ukm.npy')



print(X_train0.shape, X_final_test0.shape, y_train1.shape,y__final_test1.shape)

        


x_train11=changeX(torch.tensor(X_train0))

x_test11=changeX(torch.tensor(X_final_test0))

X_train00=x_train11
X_final_test00=x_test11

print(X_train00.shape, X_final_test00.shape, y_train1.shape,y__final_test1.shape)

X_train1=[]
X_final_test1=[]

for i in range(len(y_train1)):
    a = np.resize(X_train0[i], (3, 32, 32))
    X_train1 += [a]
    
for j in range(len(y__final_test1)):
    b = np.resize(X_final_test0[j], (3, 32, 32))
    X_final_test1 += [b]

i=0
j=0



x_train, x_test, y_train,y_test = torch.Tensor(X_train1), torch.Tensor(X_final_test1), torch.from_numpy(y_train1), torch.from_numpy(y__final_test1)

print(x_train.shape, x_test.shape, y_train.shape,y_test.shape)



train_dataset = Data.TensorDataset(x_train, y_train)
train_dataset.data = train_dataset.tensors[0]
train_dataset.targets = train_dataset.tensors[1]


test_dataset = Data.TensorDataset(x_test, y_test)
test_dataset.data = test_dataset.tensors[0]
test_dataset.targets = test_dataset.tensors[1]


labels =['ARP poisining', 'BeEF HTTP exploits','Mass HTTP requests','Metasploit exploits','Normal','Port scanning','TCP flood','UDP data flood']



train_dataset.classes = labels
test_dataset.classes = labels

train_dataset.classes_to_idx = {i: label for i, label in enumerate(labels)}
test_dataset.classes_to_idx = {i: label for i, label in enumerate(labels)}



b_s=16

trainset = Safeman_Filter(data=train_dataset.data,targets=train_dataset.targets)
print('All down Train Data:', len(trainset))
trainset.__Filter__(known=known)

#0930
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=b_s, shuffle=True,
    num_workers=4,drop_last=True)


print('Real train Data:', len(trainset))



testsetA = Safeman_Filter(data=test_dataset.data,targets=test_dataset.targets)
print('All testsetA Data:', len(testsetA))
testsetA.__Filter__(known=known)


test_loader_A = torch.utils.data.DataLoader(
    testsetA, batch_size=b_s, shuffle=False,
    num_workers=4,drop_last=True)


print('Real testsetA Data:', len(testsetA))


testsetD = Safeman_FilterD(data=test_dataset.data,targets=test_dataset.targets)
print('All testsetA Data:', len(testsetA))
testsetD.__Filter__(known=known)


test_loader_D = torch.utils.data.DataLoader(
    testsetD, batch_size=b_s, shuffle=False,
    num_workers=4,drop_last=True)


print('Real testsetD Data:', len(testsetD))


testsetC = Safeman_FilterC(data=test_dataset.data,targets=test_dataset.targets)
print('All testsetC Data:', len(testsetC))
testsetC.__Filter__(trainknown=known)


test_loader_C = torch.utils.data.DataLoader(
    testsetC, batch_size=b_s, shuffle=False,
    num_workers=4,drop_last=True)


print('Real testsetC Data:', len(testsetC))


print("done!")



==> Preparing data.............................
(10309, 46) (2578, 46) (10309,) (2578,)
len: 10309
len: 2578
torch.Size([10309, 100]) torch.Size([2578, 100]) (10309,) (2578,)
torch.Size([10309, 3, 32, 32]) torch.Size([2578, 3, 32, 32]) torch.Size([10309]) torch.Size([2578])
All down Train Data: 10309
Real train Data: 8923
All testsetA Data: 2578
Real testsetA Data: 2226
All testsetA Data: 2226
Real testsetD Data: 352
All testsetC Data: 2578
Real testsetC Data: 2578
done!


In [5]:
      
class Generatorzy(nn.Module):
    def __init__(self, z_dim):
        super(Generatorzy, self).__init__()
        
        self.fc = nn.Linear(z_dim, 256*8*8)
        self.g_deconv_1 = nn.Sequential(
                          nn.ConvTranspose2d(256, 128, kernel_size=3,
                                    stride= 2, padding=(3-2+1)//2,
                                    output_padding = (3-2)%2), 
                          nn.BatchNorm2d(128),
                          nn.LeakyReLU()
                          )
        self.g_deconv_2 = nn.Sequential(
                          nn.ConvTranspose2d(128, 64, kernel_size=3,
                                    stride= 1, padding=(3-1+1)//2,
                                    output_padding = (3-1)%2), 
                          nn.BatchNorm2d(64),
                          nn.LeakyReLU()
                          )
        self.g_deconv_3 = nn.Sequential(
                          nn.ConvTranspose2d(64, 3, kernel_size=3,
                                    stride= 2, padding=(3-2+1)//2,
                                    output_padding = (3-2)%2),
                          nn.Tanh()
                          )
        
        

    def forward(self, x):
        x = self.fc(x).view(-1, 256, 8, 8)
        x = self.g_deconv_1(x)
        x = self.g_deconv_2(x)
        x = self.g_deconv_3(x)
        
        return x     


    
class Discriminatorzy(nn.Module):
    def __init__(self):
        super(Discriminatorzy, self).__init__()
        
        self.d_conv_1 = nn.Sequential(
                          nn.Conv2d(3, 32, kernel_size=3,
                                    stride=2, padding=1), 
                          nn.LeakyReLU()
                          )
        self.d_conv_2 = nn.Sequential(
                          nn.Conv2d(32, 64, kernel_size=3,
                                    stride=2, padding=1), 
                          nn.LeakyReLU()
                          )
        self.d_conv_3 = nn.Sequential(
                          nn.Conv2d(64, 128, kernel_size=3,
                                    stride=2, padding=0), 
                          nn.LeakyReLU()
                          )
        self.fc = nn.Linear(3*3*128, 1)
        self.fczy = nn.Linear(3*3*128, 64)
        self.fcadv = nn.Linear(3*3*128, num_class)

    def forward(self, x,trainT=False):
        x = self.d_conv_1(x)
        x = self.d_conv_2(x)
        x = self.d_conv_3(x)
        x = x.view(-1, 128*3*3)
        x_zy = self.fczy(x)
        x_adv=self.fcadv(x)
        x = torch.sigmoid(self.fc(x))
        if trainT == False:
            return F.log_softmax(x_adv, dim = 1)
        else:
            return x_zy,x

In [6]:
print(device)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)     
        

z_dim=100
batch_size=b_s
discriminator = Discriminatorzy()
netDzy = discriminator.to(device)

discriminatoraux = Discriminatorzy()
netDzyaux = discriminatoraux.to(device)

generator = Generatorzy(z_dim)
netGzy = generator.to(device)


netDzy.apply(weights_init)

netDzyaux.apply(weights_init)

netGzy.apply(weights_init)

criterion = nn.BCELoss()
beta1 = 0.5

real_label = 1
fake_label = 0

optimizerD = optim.Adam(netDzy.parameters(), lr=lr/1.5, betas=(beta1, 0.999))

optimizerDaux = optim.Adam(netDzyaux.parameters(), lr=lr/1.5, betas=(beta1, 0.999))

optimizerG = optim.Adam(netGzy.parameters(), lr=lr, betas=(beta1, 0.999))

cuda:0


In [7]:


dataloader_train_closeset=train_loader
print("Starting Training Loop...")
for epoch in range(num_epochs):
    i=0
    for sample in dataloader_train_closeset:
        data, datalabel = sample
        ############################
        # (1) Update D1 D2 network
        ###########################

        netDzy.zero_grad()
        netDzyaux.zero_grad()

        real_cpu = data.to(device)

        bb_size = real_cpu.size(0)

        label = torch.full((bb_size,), real_label, device=device)
        labelaux = torch.full((bb_size,), fake_label, device=device)

        _,output = netDzy(real_cpu,trainT=True)
        output=output.view(-1)
        
        _,outputaux = netDzyaux(real_cpu,trainT=True)
        outputaux=outputaux.view(-1)

        output=output.to(torch.float32)
        outputaux=outputaux.to(torch.float32)
    
    
        label=label.to(torch.float32)
        labelaux=labelaux.to(torch.float32)
        
        errD_real = criterion(output, label)        
        errDaux_real = criterion(outputaux, labelaux)

        errD_real.backward()        
        errDaux_real.backward()
        



        noise = torch.randn(batch_size, z_dim, device=device)

        fake = netGzy(noise)
        
        label.fill_(fake_label)        
        labelaux.fill_(real_label)

        _,output = netDzy(fake.detach(),trainT=True)        
        output=output.view(-1)
        output=output.to(torch.float32)    
        
        _,outputaux = netDzyaux(fake.detach(),trainT=True)
        outputaux=outputaux.view(-1)       
        outputaux=outputaux.to(torch.float32)
        
        label=label.to(torch.float32)
        labelaux=labelaux.to(torch.float32)
        
        errD_fake = criterion(output, label)        
        errDaux_fake = criterion(outputaux, labelaux)

        errD_fake.backward()        
        errDaux_fake.backward()
        


        optimizerD.step()
        optimizerDaux.step()
        

        ############################
        # (2) Update G network
        ###########################
        netGzy.zero_grad()
        label.fill_(real_label)  
        
        labelaux.fill_(fake_label) 
        

        _,output = netDzy(fake,trainT=True)
        output=output.view(-1)

        output=output.to(torch.float32)
        label=label.to(torch.float32)
        errG = criterion(output, label)
        

        _,outputaux = netDzyaux(fake,trainT=True)
        outputaux=outputaux.view(-1)

        outputaux=outputaux.to(torch.float32)
        labelaux=labelaux.to(torch.float32)
        errGaux = criterion(outputaux, labelaux)
        


        errGqiuhe=errG+errGaux*0.1
        errGqiuhe.backward()
        
        

        optimizerG.step()

        if i % 50 == 0:
            print('[%d/%d][%d/%d]\terrD_real: %.4f\terrDaux_real: %.4f\terrD_fake: %.4f\terrDaux_fake: %.4f \terrGqiuhe: %.4f'
                  % (epoch, num_epochs, i, len(dataloader_train_closeset),
                     errD_real, errDaux_real,errD_fake,errDaux_fake, errGqiuhe))
        i+=1
        
        
    cur_model_wts = copy.deepcopy(netGzy.state_dict())
    path_to_save_paramOnly = os.path.join(save_dir, 'RPGAN-epoch-{}.GNet'.format(epoch+1))
    torch.save(cur_model_wts, path_to_save_paramOnly)
    
    cur_model_wts = copy.deepcopy(netDzy.state_dict())
    path_to_save_paramOnly = os.path.join(save_dir, 'RPGAN-epoch-{}.DNet'.format(epoch+1))
    torch.save(cur_model_wts, path_to_save_paramOnly)
    
   
    cur_model_wts = copy.deepcopy(netDzyaux.state_dict())
    path_to_save_paramOnly = os.path.join(save_dir, 'RPGAN-epoch-{}.DauxNet'.format(epoch+1))
    torch.save(cur_model_wts, path_to_save_paramOnly)

Starting Training Loop...
[0/5][0/557]	errD_real: 0.6909	errDaux_real: 0.6984	errD_fake: 0.6962	errDaux_fake: 0.6879 	errGqiuhe: 0.7605
[0/5][50/557]	errD_real: 0.6942	errDaux_real: 0.6847	errD_fake: 0.6894	errDaux_fake: 0.6917 	errGqiuhe: 0.7669
[0/5][100/557]	errD_real: 0.6788	errDaux_real: 0.6666	errD_fake: 0.6948	errDaux_fake: 0.6751 	errGqiuhe: 0.7650
[0/5][150/557]	errD_real: 0.6754	errDaux_real: 0.6352	errD_fake: 0.6851	errDaux_fake: 0.6553 	errGqiuhe: 0.7781
[0/5][200/557]	errD_real: 0.6943	errDaux_real: 0.6172	errD_fake: 0.6796	errDaux_fake: 0.6311 	errGqiuhe: 0.7847
[0/5][250/557]	errD_real: 0.6919	errDaux_real: 0.6920	errD_fake: 0.6900	errDaux_fake: 0.6974 	errGqiuhe: 0.7675
[0/5][300/557]	errD_real: 0.6961	errDaux_real: 0.7335	errD_fake: 0.6925	errDaux_fake: 0.6979 	errGqiuhe: 0.7636
[0/5][350/557]	errD_real: 0.6917	errDaux_real: 0.6859	errD_fake: 0.6933	errDaux_fake: 0.6825 	errGqiuhe: 0.7644
[0/5][400/557]	errD_real: 0.6941	errDaux_real: 0.6957	errD_fake: 0.6901	errDaux_f